In [1]:


from typing import Iterable, Mapping
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager

COL_CON = "#1C8074"
COL_SIN = "#666666"

In [2]:
notebook_name = "adipell_ensayo_guatemala"
s3 = S3AssetManager(notebook_name=notebook_name)

In [3]:

base_path = "raw/honduras"
granel = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='sin_adipel_alien_engorde')
con = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_alien_engorde')
con_punto = s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='con_adipel_punto')

sultana =  s3.read_excel(f"{base_path}/Pruebas Adipell Honduras y Salvador.xlsx", sheet_name='la_sultana_con')   

In [4]:
granel.columns = granel.columns.str.strip()
granel["fecha"] = pd.to_datetime(granel["fecha"])
granel["rendimiento"] = granel["kg producidos"]/(granel["Total minutos"]/60)

In [5]:
granel

,fecha,kg producidos,Hora inicio,Hora final,Total minutos,Ton / h,Amperaje,Presión vapor (bar),H harina,H harina acondicionada,...,T harina,T acondicionamiento,T prensa,T pellet,T ambiental,PDI,finos < 1mm,dieta,producto,rendimiento
0,2025-08-12,45670,03:59:00,06:25:00,146,18.768493,654.0,2.7,0.116,0.154,...,34.8,70.0,78.6,34.1,32.0,87.00,0.011,Aliengorde 2,sin adipell,18768.493151
1,2025-08-15,30000,04:50:00,06:00:00,70,25.714286,NaN,2.8,0.126,0.166,...,34.1,72.0,80.7,35.1,32.1,86.00,0.019,Aliengorde 2,sin adipell,25714.285714
2,2025-08-17,47110,02:46:00,05:26:00,160,17.666250,606.0,3.0,0.125,0.157,...,34.6,58.0,59.0,34.5,31.5,87.00,0.007,Aliengorde 2,sin adipell,17666.250000
3,2025-08-18,22300,07:27:00,08:37:00,70,19.114286,510.0,2.9,0.124,0.160,...,33.2,72.0,79.8,NaN,32.6,87.00,0.010,Aliengorde 2,sin adipell,19114.285714
4,NaT,24085,15:14:00,15:59:00,45,32.113333,625.0,3.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,95.32,NaN,Aliengorde 2,con adipell,32113.333333


In [6]:
granel_group = granel.groupby("producto").agg(
    toneladas_producidas=('kg producidos', 'sum'),
    tiempo_min_produccion=('Total minutos', 'sum'),
    rend_prom=('rendimiento', 'mean'),
    amperaje_promedio=('Amperaje', "mean"),
    presion_promedio=('Presión vapor (bar)', 'mean'),
    humedad_prom_harina=('H harina', "mean"),
    humedad_prom_harina_acond=('H harina acondicionada', "mean"),
    humedad_prom_harina_termidado=('H producto terminad', "mean"),
    pdi=('PDI', "mean"),
    finos=('finos < 1mm', "mean"),
).reset_index()
granel_group["tiempo_horas_produccion"] = granel_group["tiempo_min_produccion"]/60

granel_group["rendimiento"] = granel_group["toneladas_producidas"]/granel_group["tiempo_horas_produccion"]

In [7]:
granel_group["toneladas_producidas"] = granel_group["toneladas_producidas"]/1000

In [8]:
granel_group

,producto,toneladas_producidas,tiempo_min_produccion,rend_prom,amperaje_promedio,presion_promedio,humedad_prom_harina,humedad_prom_harina_acond,humedad_prom_harina_termidado,pdi,finos,tiempo_horas_produccion,rendimiento
0,con adipell,24.085,45,32113.333333,625.0,3.10,NaN,NaN,NaN,95.32,NaN,0.750000,32113.333333
1,sin adipell,145.080,446,20315.828645,590.0,2.85,0.12275,0.15925,0.125,86.75,0.01175,7.433333,19517.488789


In [24]:

def barras_1xn_tons_con_delta(
    df,
    y_cols,                       # list[str] métricas, ej: ["pdi","rendimiento","amp_prom"]
    title_prefix: str = "Comparativo de métricas",
    order=("con adipell", "sin adipell"),
    titles=None,                  # list[str] títulos por subplot; si None usa y_cols
    decimals_map=None,            # dict opcional: {"pdi":2, "rendimiento":2, ...}
):
    """
    Subplots 1×N con barras por 'producto' (con/sin adipel).
    - Etiquetas en negro.
    - 'toneladas_producidas' sobre barras si la columna existe.
    - Texto con cambio % (▲/▼) con vs sin adipel por cada subplot (si existen ambas categorías).
    - Guarda en HTML.

    Parámetros
    ----------
    df : DataFrame con columnas:
        'producto', y las métricas en y_cols. Opcional: 'toneladas_producidas'
    y_cols : list[str] métricas a graficar (una por columna del subplot)
    filename : str ruta de salida .html
    title_prefix : str título general
    order : tuple/list orden categórico en eje X
    titles : list[str] títulos individuales (longitud = len(y_cols)); si None usa y_cols
    decimals_map : dict opcional con número de decimales por métrica
    """
    d = df.copy()
    d["key"] = d["producto"].astype(str).str.strip().str.lower()
    d["color"] = d["key"].map({"con adipell": COL_CON, "sin adipell": COL_SIN})

    # ¿existe toneladas_producidas?
    has_tons = "toneladas_producidas" in d.columns

    # Título con toneladas (si existe)
    ton_text = ""
    if has_tons:
        ton_text = " · ".join([f"{p}: {t:,.0f} Ton" for p, t in zip(d["producto"], d["toneladas_producidas"])])

    n = len(y_cols)
    if titles is None:
        titles = y_cols
    if decimals_map is None:
        decimals_map = {}

    fig = make_subplots(
        rows=1,
        cols=n,
        subplot_titles=tuple(titles),
        horizontal_spacing=0.08 if n <= 3 else 0.04
    )

    def _tons(x): return f"{x:,.0f} Ton"

    def _axis_refs(col_index_1based: int):
        # ejes de datos: 'x','y' para col 1; 'x2','y2' para col>1
        xr = "x" if col_index_1based == 1 else f"x{col_index_1based}"
        yr = "y" if col_index_1based == 1 else f"y{col_index_1based}"
        # dominios: 'x domain' para col 1; 'xN domain' para col>1
        xdom = "x domain" if col_index_1based == 1 else f"x{col_index_1based} domain"
        return xr, yr, xdom

    for i, ycol in enumerate(y_cols, start=1):
        decimals = int(decimals_map.get(ycol, 2))

        # etiquetas de toneladas (solo si existe la columna)
        text_labels = [_tons(v) for v in d["toneladas_producidas"]] if has_tons else None
        text_position = "outside" if has_tons else None

        hover = (
            "<b>%{x}</b><br>"
            f"{ycol}: "+"%{y:."+str(decimals)+"f}"+ "<br>"
            + ("Toneladas: %{text}" if has_tons else "") + "<extra></extra>"
        )

        # Barras
        fig.add_trace(
            go.Bar(
                x=d["producto"],
                y=d[ycol],
                marker=dict(color=d["color"]),
                text=text_labels,
                textposition=text_position,
                textfont=dict(color="black"),
                insidetextfont=dict(color="black"),
                cliponaxis=False,
                hovertemplate=hover,
            ),
            row=1, col=i
        )

        # Texto del cambio % con vs sin adipel (si ambas categorías existen)
        if {"con adipell","sin adipell"}.issubset(set(d["key"])) and ycol in d.columns:
            try:
                v_con = float(d.loc[d["key"]=="con adipell", ycol].iloc[0])
                v_sin = float(d.loc[d["key"]=="sin adipell", ycol].iloc[0])
                if v_sin != 0 and np.isfinite(v_sin):
                    pct = (v_con - v_sin) / v_sin * 100
                    sign = "▲" if pct >= 0 else "▼"
                    note = f"{sign} {abs(pct):.1f}%"
                else:
                    note = "n/a"

                _, yref, xdom = _axis_refs(i)
                y_top = max(v_con, v_sin)
                fig.add_annotation(
                    x=0.5, xref=xdom,
                    y=y_top * 1.06, yref=yref,
                    text=note,
                    showarrow=False,
                    font=dict(color="black", size=12)
                )
            except Exception:
                # Si algo falla (valores faltantes, etc.), simplemente no ponemos el delta.
                pass

        # Orden categórico por subplot
        if order:
            fig.update_xaxes(categoryorder="array", categoryarray=list(order), row=1, col=i)

    # Layout general
    fig.update_layout(
        title_text=f"{title_prefix}",
        title_font=dict(color="black", size=18),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        showlegend=False,
        margin=dict(t=120, r=30, b=40, l=60),
        height=480,
        width=420*n if n <= 4 else 350*n  # ancho adaptable
    )

    # Ejes y grilla (aplica a todos)
    # (xaxis, xaxis2, ..., yaxis, yaxis2, ...)
    for idx in range(1, n+1):
        xax = "xaxis" if idx == 1 else f"xaxis{idx}"
        yax = "yaxis" if idx == 1 else f"yaxis{idx}"
        fig.update_layout({
            xax: dict(
                tickfont=dict(color="black"),
                titlefont=dict(color="black"),
                showline=True, linewidth=1, linecolor="black",
                mirror=True,
                gridcolor="rgba(0,0,0,0.1)"
            ),
            yax: dict(
                tickfont=dict(color="black"),
                titlefont=dict(color="black"),
                showline=True, linewidth=1, linecolor="black",
                mirror=True,
                gridcolor="rgba(0,0,0,0.1)"
            )
        })

    return fig


In [25]:
granel_group

,producto,toneladas_producidas,tiempo_min_produccion,rend_prom,amperaje_promedio,presion_promedio,humedad_prom_harina,humedad_prom_harina_acond,humedad_prom_harina_termidado,pdi,finos,tiempo_horas_produccion,rendimiento
0,con adipell,24.085,45,32113.333333,625.0,3.10,NaN,NaN,NaN,95.32,NaN,0.750000,32113.333333
1,sin adipell,145.080,446,20315.828645,590.0,2.85,0.12275,0.15925,0.125,86.75,0.01175,7.433333,19517.488789


In [26]:
barras_1xn_tons_con_delta(
    granel_group,
    y_cols=[ "rendimiento", "pdi"],
    #filename=f"{ROOT_IMAGEN}/pdi_rendimiento_granel.html",
    title_prefix="Comparativo medidas de Producción y Calidad",
    order=("con adipel", "sin adipel"),
    titles=["Rendimiento (Ton/H)", "PDI (%)"],
    decimals_map={"pdi": 2, "rendimiento": 2,}
)


In [28]:
barras_1xn_tons_con_delta(
    granel_group,
    y_cols=["amperaje_promedio", "presion_promedio"],
    #filename=f"{ROOT_IMAGEN}/amp_presion_granel.html",
    title_prefix="Comparativo parámetros de control",
    order=("con adipel", "sin adipel"),
    titles=["Amperaje promedio (Amp)", "Presion promedio (Bar)"],
    decimals_map={"amperaje_promedio": 2, "presion_promedio": 2,}
)


In [29]:

def stripplot_con_medias_pro(df, categoria: str, values: str, title: str, filename: str = None):
    """
    Strip plot profesional:
      - Puntos (semi-transparencia)
      - Boxplot translúcido (mediana + IQR)
      - Media ± IC95% por categoría
      - Línea de media global
      - Conteos n por categoría
      - Colores corporativos y orden fijo
    """

    # Orden de categorías y colores corporativos
    orden_categorias = ["dado pellet", "enfriador", "envasado"]
    colores_corporativos = {
        "dado pellet": "#1C8074",
        "enfriador": "#666666",
        "envasado": "#94AF92"
    }

    # Asegurar orden categórico
    df = df.copy()
    df[categoria] = pd.Categorical(df[categoria], categories=orden_categorias, ordered=True)

    # Estadísticas por categoría
    g = df.groupby(categoria, observed=True)[values]
    stats = g.agg(["count", "mean", "median", "std"]).reindex(orden_categorias)
    stats["sem"] = stats["std"] / np.sqrt(stats["count"].clip(lower=1))
    stats["ci95"] = 1.96 * stats["sem"]

    # Media global
    mean_global = df[values].mean()

    # 1) Strip plot (puntos)
    fig = px.strip(
        df,
        x=categoria, y=values, color=categoria,
        category_orders={categoria: orden_categorias},
        color_discrete_map=colores_corporativos,
        template="plotly_white"
    )
    # Afinar estilo de los puntos
    for tr in fig.data:
        tr.update(
            marker=dict(size=7, opacity=0.6, line=dict(width=0)),
            hovertemplate=f"<b>{values}</b>: %{{y:.2f}}<br>{categoria}: %{{x}}<extra></extra>"
        )

    # 2) Boxplot translúcido detrás (misma paleta)
    for cat in orden_categorias:
        sub = df.loc[df[categoria] == cat, values]
        if sub.empty:
            continue
        fig.add_trace(
            go.Box(
                x=[cat]*len(sub), y=sub,
                name=f"{cat} (IQR)",
                marker_color=colores_corporativos[cat],
                line=dict(color=colores_corporativos[cat]),
                fillcolor="rgba(0,0,0,0)",  # sin relleno sólido
                opacity=0.25,
                boxmean=False,
                showlegend=False
            )
        )

    # 3) Media ± IC95% por categoría (marcador + barras de error)
    fig.add_trace(
        go.Scatter(
            x=stats.index.tolist(),
            y=stats["mean"],
            mode="markers",
            name="Media ± IC95%",
            marker=dict(symbol="diamond", size=12, color="black"),
            error_y=dict(type="data", array=stats["ci95"], thickness=1.5, width=6, color="black"),
            hovertemplate="<b>Media</b>: %{y:.2f}<br>IC95%: ±%{customdata:.2f}<extra></extra>",
            customdata=stats["ci95"].values,
            showlegend=True
        )
    )

    # 4) Línea de media global
    fig.add_hline(
        y=mean_global,
        line=dict(color="black", width=1, dash="dot"),
        annotation_text=f"Media global: {mean_global:.2f}",
        annotation_position="top right",
        annotation_font_color="black"
    )

    # 5) Conteos n por categoría (debajo del eje X)
    for i, cat in enumerate(orden_categorias):
        n = int(stats.loc[cat, "count"]) if pd.notna(stats.loc[cat, "count"]) else 0
        fig.add_annotation(
            x=cat, y=df[values].min(),
            yshift=-30, showarrow=False,
            text=f"n={n}", font=dict(color="black", size=11)
        )

    # Layout pro
    fig.update_layout(
        title=title,
        xaxis_title=categoria,
        yaxis_title=values,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        margin=dict(t=80, r=30, b=80, l=60),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
    )
    fig.update_xaxes(showline=True, linecolor="black", mirror=True)
    fig.update_yaxes(showline=True, linecolor="black", mirror=True, gridcolor="rgba(0,0,0,0.1)")

    if filename:
        fig.write_html(filename, include_plotlyjs="cdn", full_html=True)

    return fig


In [30]:
orden = ["dado pellet", "enfriador", "envasado"]

stripplot_con_medias_pro(
    con_punto,
    title="Media del PDI en Aliengorde 2 con Adipel",
    categoria="punto",
    values="PDI",
    #filename=f"{ROOT_IMAGEN}/stripplot_medias_orden.html",
)


In [31]:
stripplot_con_medias_pro(
    con_punto,
    title='Media del Finos (malla 18) en Aliengorde 2 con Adipel',
    categoria="punto",
    values="Finos (malla 18)",
    #filename=f"{ROOT_IMAGEN}/stripplot_finos_18.html",
)

In [32]:

stripplot_con_medias_pro(
    con_punto,
    title='Media del Finos (malla 6) en Aliengorde 2 con Adipel',
    categoria="punto",
    values="Retencion Malla 6",
    #filename=f"{ROOT_IMAGEN}/stripplot_finos_6.html",
)

In [33]:


def stripplot_subplots_1x3_pro(
    df: pd.DataFrame,
    categoria: str,
    valores: list,          # p.ej.: ["Retencion Malla 6","Finos (malla 18)","PDI"]
    titulos: list,          # títulos de cada subplot
    orden_categorias: list, # ["dado pellet","enfriador","envasado"]
    colores: dict,          # {"dado pellet":"#1C8074","enfriador":"#666666","envasado":"#94AF92"}
    filename: str = None
):
    """
    Subplot 1x3 con strip plot + box (IQR) + media ± IC95% por categoría.
    Estilo limpio: colores corporativos, orden fijo y textos en negro.
    """
    assert len(valores) == 3 and len(titulos) == 3, "Debe ser 1x3: tres métricas y tres títulos."

    df = df.copy()
    # Asegurar orden categórico
    df[categoria] = pd.Categorical(df[categoria], categories=orden_categorias, ordered=True)

    fig = make_subplots(rows=1, cols=3, subplot_titles=tuple(titulos), horizontal_spacing=0.08)

    for i, valor in enumerate(valores, start=1):
        # --- Estadísticas por categoría ---
        g = df.groupby(categoria, observed=True)[valor]
        stats = g.agg(["count", "mean", "median", "std"]).reindex(orden_categorias)
        stats["sem"] = stats["std"] / np.sqrt(stats["count"].clip(lower=1))
        stats["ci95"] = 1.96 * stats["sem"]

        # --- 1) Box translúcido (IQR) por categoría ---
        for cat in orden_categorias:
            sub = df.loc[df[categoria] == cat, valor]
            if sub.empty:
                continue
            fig.add_trace(
                go.Box(
                    x=[cat]*len(sub),
                    y=sub,
                    name=f"{cat} (IQR)",
                    marker_color=colores[cat],
                    line=dict(color=colores[cat]),
                    fillcolor="rgba(0,0,0,0)",
                    opacity=0.25,
                    boxmean=False,
                    showlegend=False
                ),
                row=1, col=i
            )

        # --- 2) Strip plot (puntos con jitter) por categoría ---
        # Para cada categoría, desplazamos levemente en X para simular jitter
        jitter_width = 0.18
        x_pos = {cat: idx+1 for idx, cat in enumerate(orden_categorias)}  # 1..3 en el eje categórico
        for cat in orden_categorias:
            sub = df.loc[df[categoria] == cat, valor]
            if sub.empty:
                continue
            # posiciones x con jitter alrededor del índice de la categoría
            x_jitter = x_pos[cat] + (np.random.rand(len(sub)) - 0.5) * 2 * jitter_width
            fig.add_trace(
                go.Scatter(
                    x=x_jitter,
                    y=sub,
                    mode="markers",
                    marker=dict(size=7, opacity=0.6, color=colores[cat]),
                    hovertemplate=f"<b>{valor}</b>: %{{y:.2f}}<br>{categoria}: {cat}<extra></extra>",
                    showlegend=False
                ),
                row=1, col=i
            )

        # Fijar ticks categóricos correctos tras el jitter
        fig.update_xaxes(
            tickmode="array",
            tickvals=list(x_pos.values()),
            ticktext=orden_categorias,
            row=1, col=i
        )

        # --- 3) Media ± IC95% (diamante negro con error bars) ---
        fig.add_trace(
            go.Scatter(
                x=list(x_pos.values()),
                y=stats["mean"],
                mode="markers+text",
                marker=dict(symbol="diamond", size=12, color="black"),
                error_y=dict(type="data", array=stats["ci95"], thickness=1.5, width=6, color="black"),
                text=[f"{m:.2f}" if np.isfinite(m) else "" for m in stats["mean"]],
                textposition="top center",
                hovertemplate="<b>Media</b>: %{y:.2f}<br>IC95%: ±%{customdata:.2f}<extra></extra>",
                customdata=stats["ci95"].values,
                showlegend=False
            ),
            row=1, col=i
        )

    # --- Layout pro ---
    fig.update_layout(
        title="Comparativo Aliengorde 2 con Adipel",
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        margin=dict(t=90, r=30, b=70, l=60),
        height=520, width=1300
    )

    # Ejes (líneas y grilla sutil)
    for c in range(1, 4):
        fig.update_xaxes(
            showline=True, linecolor="black", mirror=True,
            row=1, col=c
        )
        fig.update_yaxes(
            showline=True, linecolor="black", mirror=True,
            gridcolor="rgba(0,0,0,0.1)",
            row=1, col=c
        )

    if filename:
        fig.write_html(filename, include_plotlyjs="cdn", full_html=True)

    return fig


In [34]:
orden_categorias = ["dado pellet", "enfriador", "envasado"]
colores = {
    "dado pellet": "#1C8074",
    "enfriador": "#666666",
    "envasado": "#94AF92"
}

stripplot_subplots_1x3_pro(
    df=con_punto,
    categoria="punto",
    valores=["Retencion Malla 6", "Finos (malla 18)", "PDI"],
    titulos=[
        "Finos (malla 6)",
        "Finos (malla 18)",
        "PDI"
    ],
    orden_categorias=orden_categorias,
    colores=colores,
    #filename=f"{ROOT_IMAGEN}/subplot_stripplots_pro.html"
)


In [35]:
sultana_group = sultana.groupby(["producto"]).agg(
    rendimiento_promedio=('Ton / h', "mean"),
    consumo_energia_promedio=('Consumo Energético kWh/T', "mean"),
    carga_promedio=("Carga", "mean"),
    presion_vapor_promedio=("Presión vapor (bar)", "mean"),
    humedad_entrada_promedio=('H harina (H entrada)', "mean"),
    humedad_acondicionador_promedio=('H harina acondicionada (Humedad salida)', "mean"),
    humedad_producto_terminado_promedio=('H producto terminado', "mean"),
    temperatura_harina_promedio=('T harina', 'mean'),
    temperatura_acondicionador_promedio=('T acondicionamiento', 'mean'),
    temperatura_peletizadora_promedio=('T pelletizadora', 'mean'),
    pdi_promedio=('PDI', 'mean'),
    finos_promedio=('finos de pellets', 'mean'),
    finos_menor_1mm_cumbre_1=('finos < 1mm crumble 1', 'mean'),
    finos_menor_1mm_cumbre_2=('finos < 1mm crumble 2', 'mean'),
    finos_mayor_1mm_menor_3_35mm_cumbre_2=('finos >1mm y < 3.35mm crumble 1', 'mean'),
    finos_mayor_3_35mm_cumbre_1=("finos > 3.35mm crumble 1", "mean"),
    finos_mayor_3_35mm_cumbre_2=("finos > 3.35mm crumble 2", "mean"),
).reset_index()

cols_finos = ['finos_promedio', 'finos_menor_1mm_cumbre_1', 'finos_menor_1mm_cumbre_2','humedad_entrada_promedio',
              'humedad_acondicionador_promedio', 'humedad_producto_terminado_promedio', 'finos_mayor_1mm_menor_3_35mm_cumbre_2', 'finos_mayor_3_35mm_cumbre_1', 'finos_mayor_3_35mm_cumbre_2']
for col in cols_finos:
    sultana_group[col] = sultana_group[col]*100
sultana_group

,producto,rendimiento_promedio,consumo_energia_promedio,carga_promedio,presion_vapor_promedio,humedad_entrada_promedio,humedad_acondicionador_promedio,humedad_producto_terminado_promedio,temperatura_harina_promedio,temperatura_acondicionador_promedio,temperatura_peletizadora_promedio,pdi_promedio,finos_promedio,finos_menor_1mm_cumbre_1,finos_menor_1mm_cumbre_2,finos_mayor_1mm_menor_3_35mm_cumbre_2,finos_mayor_3_35mm_cumbre_1,finos_mayor_3_35mm_cumbre_2
0,con adipell,17.62,11.370,0.7280,1.7,12.881667,14.7150,12.5350,31.7,81.833333,82.166667,91.581667,2.548333,18.196667,16.751667,61.7650,20.038333,20.836667
1,sin adipell,17.20,11.575,0.7175,1.7,12.910000,15.1275,12.6125,33.2,82.500000,82.000000,92.070000,1.587500,17.862500,16.387500,60.1225,22.132500,23.172500


In [36]:
barras_1xn_tons_con_delta(
    sultana_group,
    y_cols=["rendimiento_promedio",  "pdi_promedio", 'finos_promedio',],
    #filename=f"{ROOT_IMAGEN}/pdi_finos_rendimiento_sultana.html",
    title_prefix="Comparativo medidas de Producción y Calidad",
    order=("con adipell", "sin adipell"),
    titles=["Rendimiento (Ton/h)", "PDI (%)", 'Finos (%)', ],
    decimals_map={"pdi_promedio": 2, 'finos_promedio': 2, "rendimiento_promedio": 2,}
)


In [37]:
barras_1xn_tons_con_delta(
    sultana_group,
    y_cols=["consumo_energia_promedio", 'carga_promedio', 'presion_vapor_promedio'],
    #filename=f"{ROOT_IMAGEN}/consumo_energia_carga_sultana.html",
    title_prefix="Comparativo medidas de Control",
    order=("con adipell", "sin adipell"),
    titles=["Consumo Energético Promedio (kWh/T)",'Carga Promedio (%)',  "Presión vapor Promedio (bar)"],
    decimals_map={"pdi_promedio": 2, 'finos_promedio': 2, "rendimiento_promedio": 2,}
)

In [38]:
sultana_group

,producto,rendimiento_promedio,consumo_energia_promedio,carga_promedio,presion_vapor_promedio,humedad_entrada_promedio,humedad_acondicionador_promedio,humedad_producto_terminado_promedio,temperatura_harina_promedio,temperatura_acondicionador_promedio,temperatura_peletizadora_promedio,pdi_promedio,finos_promedio,finos_menor_1mm_cumbre_1,finos_menor_1mm_cumbre_2,finos_mayor_1mm_menor_3_35mm_cumbre_2,finos_mayor_3_35mm_cumbre_1,finos_mayor_3_35mm_cumbre_2
0,con adipell,17.62,11.370,0.7280,1.7,12.881667,14.7150,12.5350,31.7,81.833333,82.166667,91.581667,2.548333,18.196667,16.751667,61.7650,20.038333,20.836667
1,sin adipell,17.20,11.575,0.7175,1.7,12.910000,15.1275,12.6125,33.2,82.500000,82.000000,92.070000,1.587500,17.862500,16.387500,60.1225,22.132500,23.172500


In [39]:

def grafico_lineas_generalizado(
    df,
    x_cols,
    x_labels,
    filename=None,
    title="Comparación de Temperaturas",
    y_title="Temperatura (°C)",
    series_col="producto",
    series_names=("con adipell", "sin adipell"),
    colors=("#1C8074", "#666666")
):
    """
    Genera un gráfico de líneas con puntos en forma de X para múltiples series.

    Parámetros:
    -----------
    df : DataFrame con los datos.
    x_cols : list[str] columnas del DataFrame para el eje X.
    x_labels : list[str] etiquetas del eje X (mismo orden que x_cols).
    filename : str, opcional, si se pasa guarda el gráfico en HTML.
    title : str, título del gráfico.
    y_title : str, etiqueta del eje Y.
    series_col : str, columna con los nombres de las series (ej. 'producto').
    series_names : tuple[str], nombres de las series a graficar.
    colors : tuple[str], colores para las series.
    """
    fig = go.Figure()

    for serie, color in zip(series_names, colors):
        valores = df.loc[df[series_col] == serie, x_cols].values.flatten()
        fig.add_trace(go.Scatter(
            x=x_labels,
            y=valores,
            mode="lines+markers+text",
            name=serie,
            line=dict(color=color, width=2),
            marker=dict(symbol="x", size=10, color=color),  # Puntos en forma de X
            text=[f"{v:.1f}" for v in valores],
            textposition="top center"
        ))

    # Estilo general
    fig.update_layout(
        title=title,
        xaxis_title="Etapa del proceso",
        yaxis_title=y_title,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font=dict(color="black"),
        showlegend=True,
        margin=dict(t=80, r=40, b=40, l=60)
    )

    if filename:
        fig.write_html(filename, include_plotlyjs="cdn", full_html=True)

    return fig


In [40]:
x_cols = [
    "temperatura_harina_promedio",
    "temperatura_acondicionador_promedio",
    "temperatura_peletizadora_promedio"
]
x_labels = ["Harina", "Acondicionador", "Peletizadora"]

grafico_lineas_generalizado(

    sultana_group,
    title="Comparación de Temperaturas por punto de medida",

    x_cols=x_cols,
    x_labels=x_labels,
    #filename=f"{ROOT_IMAGEN}/lineas_temperaturas.html"
)


In [41]:
x_cols = [
    "humedad_entrada_promedio",
    "humedad_acondicionador_promedio",
    "humedad_producto_terminado_promedio"
]
x_labels = ["Humedad Entrada", "Humedad Acondicionador", "Humedad Producto Terminado"]

grafico_lineas_generalizado(
    sultana_group,
    title="Comparación de Humedades por punto de medida",
    y_title="Humedad (%)",
    x_cols=x_cols,
    x_labels=x_labels,
    #filename=f"{ROOT_IMAGEN}/lineas_humedades.html"
)


In [42]:
x_cols = [
    "finos_menor_1mm_cumbre_2",
    "finos_mayor_1mm_menor_3_35mm_cumbre_2",
    "finos_mayor_3_35mm_cumbre_2"
]
x_labels = ["Finos menores 1mm", "Finos entre 1mm y 3.35mm", "Finos mayores a 3,35mm"]

grafico_lineas_generalizado(
    sultana_group,
    title="Comparación de finos en el cumbre 2",
    y_title= "Finos (%)",
    x_cols=x_cols,
    x_labels=x_labels,
    #=f"{ROOT_IMAGEN}/finos_cumbre_2.html"
)
